In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout,GlobalMaxPooling1D
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix, roc_curve, auc

In [35]:
data=pd.read_csv(r'C:\Users\sujay\Desktop\Chatbot\CSVFiles\New_Hinglish_Classification_final_1.csv')

In [36]:
data

,english,Word,label
0,0,cricket,en
1,1,chal,hi
2,2,raha,hi
3,3,hain,hi
4,4,yaha,hi
...,...,...,...
12744,12744,challenge,en
12745,12745,winners,en
12746,12746,at,en
12747,12747,paradise,en


In [37]:
data.drop(['english'],axis=1,inplace=True)

In [38]:
data

,Word,label
0,cricket,en
1,chal,hi
2,raha,hi
3,hain,hi
4,yaha,hi
...,...,...
12744,challenge,en
12745,winners,en
12746,at,en
12747,paradise,en


In [39]:
data.replace({"label": {"en": 1, "hi": 0}}, inplace=True)

C:\Users\sujay\AppData\Local\Temp\ipykernel_11988\1330269346.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"label": {"en": 1, "hi": 0}}, inplace=True)


In [40]:
data

,Word,label
0,cricket,1
1,chal,0
2,raha,0
3,hain,0
4,yaha,0
...,...,...
12744,challenge,1
12745,winners,1
12746,at,1
12747,paradise,1


In [41]:
"""
split_ratios = {
    "80-20": 0.2,
    "99-1": 0.01,
    "75-25": 0.25,
    "60-40": 0.4
    }
results=[]
for split_name, test_size in split_ratios.items():
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=42)
    tokenizer = Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(train_data["Word"])
    X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["Word"]), maxlen=200)
    X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["Word"]), maxlen=200)
    Y_train = train_data["label"].values
    Y_test = test_data["label"].values    
    X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["Word"]), maxlen=200)
    X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["Word"]), maxlen=200)
    Y_train = train_data["label"].values
    Y_test = test_data["label"].values
    model = Sequential([
    Embedding(input_dim=10000, output_dim=128),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1)
    y_pred_prob = model.predict(X_test).flatten()
    y_pred_classes = (y_pred_prob > 0.5).astype(int)
    precision = precision_score(Y_test, y_pred_classes)
    recall = recall_score(Y_test, y_pred_classes)
    f1 = f1_score(Y_test, y_pred_classes)
    accuracy = accuracy_score(Y_test, y_pred_classes)
    results.append({
        "Split": split_name,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Accuracy": accuracy
    })
    if split_name == "80-20":
        # Confusion Matrix
        print("Confusion Matrix and ROC Curve for 80-20 Split:")
        cm = confusion_matrix(Y_test, y_pred_classes)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix (80-20 Split)')
        plt.show()
        fpr, tpr, _ = roc_curve(Y_test, y_pred_prob)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve (80-20 Split)')
        plt.legend(loc="lower right")
        plt.show()
    if split_name == "99-1":
        print("Confusion Matrix and ROC Curve for 99-1 Split:")
        cm = confusion_matrix(Y_test, y_pred_classes)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix (99-1 Split)')
        plt.show()
        fpr, tpr, _ = roc_curve(Y_test, y_pred_prob)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve (99-1 Split)')
        plt.legend(loc="lower right")
        plt.show()
    if split_name == "75-25":
        print("Confusion Matrix and ROC Curve for 75-25 Split:")
        cm = confusion_matrix(Y_test, y_pred_classes)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix (75-25 Split)')
        plt.show()
        fpr, tpr, _ = roc_curve(Y_test, y_pred_prob)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve (75-25 Split)')
        plt.legend(loc="lower right")
        plt.show()
    if split_name == "60-40":
        print("Confusion Matrix and ROC Curve for 60-40 Split:")
        cm = confusion_matrix(Y_test, y_pred_classes)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix (60-40 Split)')
        plt.show()
        fpr, tpr, _ = roc_curve(Y_test, y_pred_prob)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve (60-40 Split)')
        plt.legend(loc="lower right")
        plt.show()
results_df = pd.DataFrame(results)
print("Evaluation Metrics for Different Splits:")
print(results_df)
"""

'\nsplit_ratios = {\n    "80-20": 0.2,\n    "99-1": 0.01,\n    "75-25": 0.25,\n    "60-40": 0.4\n    }\nresults=[]\nfor split_name, test_size in split_ratios.items():\n    train_data, test_data = train_test_split(data, test_size=test_size, random_state=42)\n    tokenizer = Tokenizer(num_words=10000)\n    tokenizer.fit_on_texts(train_data["Word"])\n    X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["Word"]), maxlen=200)\n    X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["Word"]), maxlen=200)\n    Y_train = train_data["label"].values\n    Y_test = test_data["label"].values    \n    X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["Word"]), maxlen=200)\n    X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["Word"]), maxlen=200)\n    Y_train = train_data["label"].values\n    Y_test = test_data["label"].values\n    model = Sequential([\n    Embedding(input_dim=10000, output_dim=128),\n    LSTM(128, dropout=0.2, recurrent_dropout=0.

In [42]:
#75-25 gives the bestt result

In [43]:
data

,Word,label
0,cricket,1
1,chal,0
2,raha,0
3,hain,0
4,yaha,0
...,...,...
12744,challenge,1
12745,winners,1
12746,at,1
12747,paradise,1


In [44]:

results_2=[]
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_data["Word"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["Word"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["Word"]), maxlen=200)
Y_train = train_data["label"].values
Y_test = test_data["label"].values    
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["Word"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["Word"]), maxlen=200)
Y_train = train_data["label"].values
Y_test = test_data["label"].values
model = Sequential([
Embedding(input_dim=10000, output_dim=128),
LSTM(128, dropout=0.2, recurrent_dropout=0.2),
Dense(1, activation="sigmoid")
])
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_split=0.3, verbose=1)
y_pred_prob = model.predict(X_test).flatten()
y_pred_classes = (y_pred_prob > 0.5).astype(int)
precision = precision_score(Y_test, y_pred_classes)
recall = recall_score(Y_test, y_pred_classes)
f1 = f1_score(Y_test, y_pred_classes)
accuracy = accuracy_score(Y_test, y_pred_classes)
results_2.append({
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1,
    "Accuracy": accuracy
})

Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 18s 158ms/step - accuracy: 0.5686 - loss: 0.6640 - val_accuracy: 0.8045 - val_loss: 0.4159
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 16s 152ms/step - accuracy: 0.9369 - loss: 0.2681 - val_accuracy: 0.8045 - val_loss: 0.3483
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 16s 155ms/step - accuracy: 0.9700 - loss: 0.0920 - val_accuracy: 0.7996 - val_loss: 0.3590
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 16s 150ms/step - accuracy: 0.9732 - loss: 0.0714 - val_accuracy: 0.8006 - val_loss: 0.3522
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 15s 147ms/step - accuracy: 0.9725 - loss: 0.0591 - val_accuracy: 0.8020 - val_loss: 0.3543
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 16s 156ms/step - accuracy: 0.9746 - loss: 0.0540 - val_accuracy: 0.8045 - val_loss: 0.3484
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 19s 179ms/step - accuracy: 0.9744 - loss: 0.0558 - val_accuracy: 0.7933 - val_loss: 0.3518
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 18s 172ms/step - accuracy: 0.9780 - loss: 0

In [45]:
print(results_2)

[{'Precision': 0.7634973721930244, 'Recall': 0.9506246281975015, 'F1 Score': 0.8468468468468469, 'Accuracy': 0.8186951066499373}]


In [46]:
def predict_label(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "en" if prediction[0][0] > 0.5 else "hi"
    return sentiment

In [47]:
review="bhai"
print(predict_label(review))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
hi


In [48]:
with open(r'C:\Users\sujay\Desktop\Chatbot\models\tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [49]:
model.save(r'C:\Users\sujay\Desktop\Chatbot\models\Hinglish_Classification_LSTM_Model.h5')